# Unet-Pytorch
Este proyecto nace con la idea de tener una alternativa a la implementación de U-Net en Tensorflow. La idea es que sea lo más parecida posible a la implementación de Tensorflow, pero en Pytorch.

## Importar librerías

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os, random, torch, shutil, cv2
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms as T
from torch.utils.data import random_split, DataLoader
import glob

import PIL, skimage.io
from PIL import Image

from lxml import etree


## Comprobar si hay GPU

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Directorios

In [21]:
# Localización de los frames
annotation_file = "data/annotations.xml"
frames_dir = "data/frames/"
train_location = "data/train/*"
mask_location = "data/mask/*"
frames = os.listdir(frames_dir)

print(len(frames))
labels = {"acera":0,
          "ciclovia":0,
          "cruceZebra":0}


2892


In [6]:
frames[:5]

['calzada_galvan_0.jpg',
 'calzada_galvan_1.jpg',
 'calzada_galvan_10.jpg',
 'calzada_galvan_100.jpg',
 'calzada_galvan_1000.jpg']

## Limpiar directorios 

In [5]:
def clean_dir(path):
    files = os.listdir(path)
    if files:
        for f in files:
            os.remove(os.path.join(path, f))
        print(path, "Limpio")
    else:
        print(path,"Actualmente vacia")
        
def cleanProcess():
    #Limpiamos los directorios
    clean_dir(train_location)
    clean_dir(mask_location)
    clean_dir("data/frameSinEtiquetar")
    
cleanProcess()

data/train/ Limpio
data/mask/ Limpio
data/frameSinEtiquetar Limpio


## Copiar frames a directorios

In [10]:
def drop_traslapes(mask):
    for d in range(mask.shape[2]-1):
        matriz_b_existe = False
        matrix_binaria = None

        if not matriz_b_existe:
            if len(np.unique(matrix_binaria))==0:
                continue
            else:
                matrix_binaria = mask[:,:,d].copy()
                matriz_b_existe = True

        matrix_binaria[matrix_binaria == 0] = 1
        matrix_binaria[matrix_binaria == 255] = 0

        mask[:,:,d+1] = matrix_binaria * mask[:,:,d+1]
    return mask

def create_mask_file(width = 1920, height = 1080, annotation=False):
  """
  Función para enmascarar las imagenes con las etiquetas
  :param width:
  :param height:
  :param annotation:
  :return:
  """
  labels = {"acera":0,"ciclovia":1,"cruceZebra":2}

  m = np.full((height, width) ,0, dtype=np.uint8)
  rgb = [m,m,m] # red, green, blue

  for shape in annotation[0]["shapes"]:
    mask = np.full((height, width) ,0, dtype=np.uint8)
    label = labels[shape["label"]]
    points = [tuple(map(float, p.split(','))) for p in shape['points'].split(';')]
    points = np.array([(int(p[0]), int(p[1])) for p in points])
    points = points.astype(int)
    mask = cv2.drawContours(mask, [points], -1, 255)
    mask = cv2.fillPoly(mask, [points], color=255)
    rgb[label] = mask
  return np.dstack(rgb)

def readAnnotations(cvat_xml, image_name):
    """
    Función para extraer los datos de cada annotation
    :param cvat_xml:
    :param image_name:
    :return:
    """
    root = etree.parse(cvat_xml).getroot()
    anno = []
    image_name_attr = ".//image[@name='{}']".format(image_name)
    for image_tag in root.iterfind(image_name_attr):
      image = {}
      for key, value in image_tag.items():
        image[key] = value
      image['shapes'] = []
      for poly_tag in image_tag.iter('polygon'):
        polygon = {'type': 'polygon'}
        for key, value in poly_tag.items():
            polygon[key] = value
        image['shapes'].append(polygon)
      for box_tag in image_tag.iter('box'):
        box = {'type': 'box'}
        for key, value in box_tag.items():
            box[key] = value
        box['points'] = "{0},{1};{2},{1};{2},{3};{0},{3}".format(
            box['xtl'], box['ytl'], box['xbr'], box['ybr'])
        image['shapes'].append(box)
      image['shapes'].sort(key=lambda x: int(x.get('z_order', 0)))
      anno.append(image)
    return anno
  
def moveFrames():
  for frame in frames:
    if ').jpg' in frame:
        continue
      
    annotation = readAnnotations(annotation_file, frame)
    if annotation[0]["shapes"]: #Validar que el frame esté etiquedato
        file_labeled = annotation[0]["name"]

        # frames
        actualLoc_frame = os.path.join(frames_dir, file_labeled)
        destination_frame = os.path.join(train_location, file_labeled)
        destination_mask  = os.path.join(mask_location, frame[:-4]+".tif")
        shutil.copy(actualLoc_frame, destination_frame)
        
        # mascaras
        mask = create_mask_file(annotation=annotation)
        mask = drop_traslapes(mask)
        
        # Guardar la mascara en destination_mask como jpg
        skimage.io.imsave(destination_mask, mask, plugin="tifffile")
        
        for img in annotation:
            for shape in img["shapes"]:
                labels[shape["label"]] += 1
    else:
        file_unlabeled = annotation[0]["name"]
        actualLoc_frame = os.path.join(frames_dir, file_unlabeled)
        destination_frame = os.path.join("data/frameSinEtiquetar", file_unlabeled)
        shutil.copy(actualLoc_frame, destination_frame)

moveFrames()
print("Proceso de copiado... Completo")

print(labels)
#input("Presione Enter para continuar...")

C:\Users\pedro\AppData\Local\Temp\ipykernel_5024\1630754020.py:92: UserWarning: data/mask/calzada_galvan_1044.tif is a low contrast image
  skimage.io.imsave(destination_mask, mask, plugin="tifffile")
C:\Users\pedro\AppData\Local\Temp\ipykernel_5024\1630754020.py:92: UserWarning: data/mask/calzada_galvan_1052.tif is a low contrast image
  skimage.io.imsave(destination_mask, mask, plugin="tifffile")
C:\Users\pedro\AppData\Local\Temp\ipykernel_5024\1630754020.py:92: UserWarning: data/mask/calzada_galvan_1062.tif is a low contrast image
  skimage.io.imsave(destination_mask, mask, plugin="tifffile")
C:\Users\pedro\AppData\Local\Temp\ipykernel_5024\1630754020.py:92: UserWarning: data/mask/calzada_galvan_1070.tif is a low contrast image
  skimage.io.imsave(destination_mask, mask, plugin="tifffile")
C:\Users\pedro\AppData\Local\Temp\ipykernel_5024\1630754020.py:92: UserWarning: data/mask/calzada_galvan_1075.tif is a low contrast image
  skimage.io.imsave(destination_mask, mask, plugin="tifffi

Proceso de copiado... Completo
{'acera': 2855, 'ciclovia': 1183, 'cruceZebra': 700}


## Cargamos los directorios de train y mask

In [22]:
train_frames = glob.glob(train_location)
print(len(train_frames))
train_frames[:5]

2728


['data/train\\calzada_galvan_10.jpg',
 'data/train\\calzada_galvan_100.jpg',
 'data/train\\calzada_galvan_1000.jpg',
 'data/train\\calzada_galvan_1001.jpg',
 'data/train\\calzada_galvan_1002.jpg']

In [24]:
mask_frames = glob.glob(mask_location)
print(len(mask_frames))
mask_frames[:5]

2728


['data/mask\\calzada_galvan_10.tif',
 'data/mask\\calzada_galvan_100.tif',
 'data/mask\\calzada_galvan_1000.tif',
 'data/mask\\calzada_galvan_1001.tif',
 'data/mask\\calzada_galvan_1002.tif']

## Creación del dataset

In [25]:
class Unet_Dataset(Dataset):
  def __init__(self, data, masks=None, img_transform=None, mask_transform=None):
    '''
    data - train data path
    masks - train masks path
    '''
    self.data = data
    self.masks = masks
    
    self.img_transform = img_transform
    self.mask_transform = mask_transform
    
    self.images = sorted(os.listdir(self.train_data))
    self.masks = sorted(os.listdir(self.train_masks))
    
  def __len__(self):
    if self.train_masks is not None:
      assert len(self.images) == len(self.masks), "Images and masks must have the same length"
    return len(self.images)
  
  def __getitem__(self, idx):
    image_name = os.path.join(self.train_data, self.images[idx])

NameError: name 'Dataset' is not defined